# Process SDII Data with GeoPandasAdapter

**This notebook demonstrates how to read and perform basic exploratory analysis on archived and streaming SDII data using the Python SDK. SDII stands for Sensor Data Ingestion Interface. [Learn more about the SDII format specification](https://developer.here.com/documentation/sdii-data-spec/topics/introduction.html "SDII Documentation")**


### Dependencies
* Catalogs: [SDII Sample Berlin](https://platform.here.com/data/hrn:here:data::olp-here:olp-sdii-sample-berlin-2)  
* Layers: [Sample Versioned Layer](https://platform.here.com/data/hrn:here:data::olp-here:olp-sdii-sample-berlin-2/sample-versioned-layer)
* Languages: Python


### Test data sample description
The data sample contains simulated vehicle travel paths and corresponding simulated traffic sign observations. Although the schema represents the actual SDII data structure not all fields have been populated. The simulated signs are for demonstration purposes only and do not represent actual sign placements. 

### Workflow
- Import Python packages
- Import required Python SDK libraries
- Specify data catalog and layer
- Specify a bounding box for the data and get the corresponding tiles
- Read SDII data from a versioned layer and deserialize using the Python SDK utilities
- Extract information of interest to _pandas_ dataframes for manipulation and visualization
- Perform simple exploratory analysis
- Read SDII data from a streaming layer and print some trace information

## Import Python Packages

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd

## Import Required Python SDK libraries

In [ ]:
from here.platform import Platform
from here.geotiles import heretile as ht
from here.geopandas_adapter import GeoPandasAdapter
from here.geopandas_adapter.utils.dataframe import unpack_columns
from here.inspector import inspect, Color

In [ ]:
platform = Platform(adapter=GeoPandasAdapter())

## Specify data catalog and layer

In [ ]:
CATALOG_HRN = 'hrn:here:data::olp-here:olp-sdii-sample-berlin-2'
LAYER_ID = 'sample-versioned-layer'

catalog = platform.get_catalog(CATALOG_HRN)
layer = catalog.get_layer(LAYER_ID)

## Specify a bounding box for the data and get the corresponding tiles

In [ ]:
tile_ids = list(ht.in_bounding_box(
            west=13.3,
            south=52.5,
            east=13.5,
            north=52.6,
            level=14,
            fully_contained=False))
tile_ids[:10]

## Read SDII data from a versioned layer and deserialize using the Python SDK utilities

Each message contains zero or more events. We're interested in `signRecognition` events and `positionEstimate`. We'll explode the list and unpack the dictionaries contained in the messages in subsequent cells.

In [ ]:
parsed_blobs = layer.read_partitions(tile_ids, record_path="messages", columns=["messageId", "message.pathEvents.signRecognition", "message.path.positionEstimate"])
parsed_blobs

### Number of drives per archiving tile - drives are associated with tiles based on the first GPS point in the trace

In [ ]:
drives_per_tile = parsed_blobs['partition_id'].value_counts()
drives_per_tile

## Traffic sign information

### Statistics for different sign types in the sample

Given the nested nature of the `signRecognition`, we explode the column and unpack the resulting dictionaries, thanks to the `explode` and `unpack_columns` helper function. `explode` is a `pd.DataFrame` function to turn list in multiple rows. `unpack_columns` is a convenience function provided by HERE to turn dictionaries into independent columns.

In [ ]:
exploded_signs = parsed_blobs[["messageId", "message.pathEvents.signRecognition"]].explode("message.pathEvents.signRecognition")
exploded_signs

`unpack_columns` exposes the fields and subfields of dictionaries as independent columns. The `keep_prefix=False` parameter removes the prefix `message.pathEvents.signRecognition` from the column names, for convenience.

In [ ]:
signs = unpack_columns(exploded_signs, "message.pathEvents.signRecognition", keep_prefix=False)
signs

In [ ]:
sign_by_type = signs.groupby('roadSignType').count()
sign_by_type = sign_by_type[["messageId"]].rename(columns={'messageId':'count'})
sign_by_type.head()

### Statistics for different sign types in the sample

In [ ]:
plt.rcParams['figure.figsize'] = (20, 6)
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18

In [ ]:
cmap = plt.get_cmap("tab20c")

labels_signs =  sign_by_type.index

pie = plt.pie(sign_by_type['count'], shadow=False, startangle=140, pctdistance=0.85, colors=cmap.colors)

plt.legend(pie[0], labels_signs, bbox_to_anchor=(1,0.5), loc="center right", fontsize=12,
           bbox_transform=plt.gcf().transFigure)

plt.tight_layout()

### Number of signs by permanency

In [ ]:
sign_count_by_permanency = signs.groupby('roadSignPermanency').count()
sign_count_by_permanency = sign_count_by_permanency[["messageId"]].rename(columns = {'messageId': 'count'})
sign_count_by_permanency

### Number of observation of speed limit signs by value

In [ ]:
sign_count_by_value = signs[signs['roadSignType'] == "SPEED_LIMIT_START"].groupby('roadSignValue').count()
sign_count_by_value = sign_count_by_value[["messageId"]].rename(columns = {'messageId': 'count'})

sign_count_by_value['speed'] = sign_count_by_value.index

plt.bar(sign_count_by_value['speed'], sign_count_by_value['count'])
plt.xlabel('Speed sign value')
plt.ylabel('Number of signs')
plt.grid(True)

### Number of GPS points recorded in each drive (sorted from longest to shortest trace)

In [ ]:
exploded_gps = parsed_blobs[["messageId", "message.path.positionEstimate"]].explode("message.path.positionEstimate")
exploded_gps

In [ ]:
gps = unpack_columns(exploded_gps, "message.path.positionEstimate", keep_prefix=False)
gps

In [ ]:
grouped_data = gps.groupby('messageId').count()
msgs_by_count = grouped_data.rename(columns = {'timeStampUTC_ms': 'count'})

max_obs = msgs_by_count['count'].max()

msgs_by_count['count'].hist(bins=list(range(1,max_obs+2, 5)))

plt.title('Number of GPS points per message')
plt.xlabel('Number of GPS points')
plt.ylabel('Number of messages')
plt.grid(True)

### Turning the DataFrame into a GeoDataFrame

Using _geopandas_, we can easily render and perform geographic operations on data. To enable this, we need to create a geometry column: in this case we take the `longitude_deg` and `latitude_deg` columns and turn them into a _geopandas_ geometry column. For clarity, we then select only columns we use further.

In [ ]:
gps_geo = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps["longitude_deg"], gps["latitude_deg"]))
gps_geo = gps_geo[["messageId", "geometry"]]
gps_geo

### Get and render the longest drive (based on number of GPS points)

In [ ]:
longest_msg = gps_geo['messageId'].value_counts().idxmax()
longest_gps = gps_geo[gps_geo['messageId'] == longest_msg]
longest_gps

In [ ]:
inspect(longest_gps, style=Color.BLUE)

## Loading SDII data from streaming layers

In [ ]:
LAYER_ID = "sample-streaming-layer"

layer = catalog.get_layer(LAYER_ID)
subscription = layer.subscribe()

In [ ]:
parsed_msgs = layer.read_stream(subscription, columns=["path.positionEstimate"])
parsed_msgs

In this second example, we want to obtain line geometry directly from lists contained in `path.positionEstimate`. Since those lists are in a custom format, the best way we can achieve our goal is using a custom function applied to the column. The alternative would be exploding the list, unpacking the columns, converting `longitude_deg` and `latitude_deg` columns to points, re-grouping by message identifier, turning the list of points to a line geometry. Sometimes, direct ways enabled by custom functions are more comfortable.

In [ ]:
from shapely.geometry import LineString

def to_linestring(estimates):
    coords = [(e["longitude_deg"], e["latitude_deg"]) for e in estimates] 
    return LineString(coords)

pos_geo = gpd.GeoDataFrame(parsed_msgs, geometry=parsed_msgs["path.positionEstimate"].apply(to_linestring))
pos_geo = pos_geo[["partition_id", "geometry"]]
pos_geo

### Plot the coordinates

In [ ]:
inspect(pos_geo, style=Color.RED)

### Cleaning up

When done, unsubscribe from the stream.

In [ ]:
subscription.unsubscribe()

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>